Before you turn this lab in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE".

**Provide your name and any collaborators below:**

In [ ]:
// YOUR CODE HERE
???

# Lab 6 - Formal Verification
> Labs will be due each week before the homework. They are not intended take a significant amount of time but rather to provide examples/practice on specific and isolated features in the language.

**This lab has reduced autograding functionality!** The staff will need to manually grade inspect some portions of this lab. Thus, please be sure to check (and double check) your work as best as you can before submitting. The autograder score may not be the final score.

When completing this lab, you may frequently find the assertions you write fail. Often times, you need to tweak your assertion to better consider corner cases. To best understand what is causing your assertion to fail, be sure to use the additional arguments to `assert` to print out the values and an informative string.

### Import the necessary Chisel dependencies. 
> There will be a cell like this in every lab. Make sure you run it before proceeding to bring the Chisel Library into the Jupyter Notebook scope!

In [ ]:
interp.load.module(os.Path(s"${System.getProperty("user.dir")}/resource/chisel_deps.sc"))

In [ ]:
import chisel3._
import chisel3.util._
import chiseltest._
import chiseltest.formal._
import chiseltest.RawTester._

## Problem 1 - Majority Circuit (Combinational Logic) (6 points)
> The `Majority` generator generates a [majority circuit](https://en.wikipedia.org/wiki/Majority_function) for a given number of inputs. A majority circuit returns true if greater than half of its inputs are true. Although it is implemented, the implementation is non-trivial, so we would like to formally verify it.

In [ ]:
class Majority(numInputs: Int) extends Module {
    val io = IO(new Bundle {
        val in = Input(Vec(numInputs, Bool()))
        val out = Output(Bool())
    })
    val majorityThreshold = (numInputs / 2) + 1
    val allPossibleWins = (0 until numInputs).combinations(majorityThreshold)
    val allAndGates = allPossibleWins.map{
        combo => combo.map(index => io.in(index)).reduce(_ && _)
    }
    io.out := allAndGates.reduce(_ || _)
}

> When writing assertions, it is sometimes nice to declare additional Chisel components to track data (via registers) or pull complicated expressions out of the assertion statements. To prevent this additional hardware from adding onto the original module, we can make a new module that extends the original module (via inheritance). We can put these assertions and their corresponding extra hardware in this child module, leaving the original module untouched. The child module has access to all of the internals of the original (parent) module. This access to internals (via inheritance) is an advantage of this child approach instead of making a simple module to wrap the original module.

> Fill in the `MajorityChecker` module below to check the majority functionality. You will need to do two things:
> 1. Implement the majority functionality via an alternate mthod. We recommend using the [PopCount](https://javadoc.io/doc/edu.berkeley.cs/chisel3_2.13/latest/chisel3/util/PopCount$.html) functionality after converting the input to a `UInt` (via `.asUInt`).
> 2. Write an assertion to check that result (`io.out`) is always equal to your alternate implementation.

In [ ]:
class MajorityChecker(numInputs: Int) extends Majority(numInputs) {
    // YOUR CODE HERE
    ???
}

In [ ]:
for (n <- 3 to 5)
    verify(new MajorityChecker(n), Seq(BoundedCheck(1)))

## Problem 2 - Bounded Counter (Simple State Machine) (7 points)
> In the problem, you will gain familiarity with doing temporal assertions. The `past` function lets you reference the value of a signal on the prior cycle. For example, `x === past(x)` checks if the signal `x` is the same as it was last cycle.

> The `BoundedCounter` module below wraps Chisel's Counter generator. Write an assertion to check that it increments. Specifically, check that if in the last cycle `io.enable` was high, the current value of `count` is one larger. _Hint:_ you will need to consider (or exclude) the corner case of wrap-around.

In [ ]:
class BoundedCounter(max: Int) extends Module {
    require(max > 0)
    val io = IO(new Bundle {
        val enable = Input(Bool())
        val count = Output(Bool())
    })
    val (count, wrap) = Counter(io.enable, max)
    io.count := count

    // YOUR CODE HERE
    ???
    
}

In [ ]:
verify(new BoundedCounter(8), Seq(BoundedCheck(10)))

## Problem 3 - Greatest Common Divisor (Complicated State Machine) (12 points)
> The `GCD` module below uses the [Euclidean GCD Algorithm](https://en.wikipedia.org/wiki/Euclidean_algorithm) and is based on code originally from the [chisel-tutorial](https://github.com/ucb-bar/chisel-tutorial).

In [ ]:
object GCDState extends ChiselEnum {
    val empty, running, done = Value
}

class GCD(w: Int) extends Module {
    val io = IO(new Bundle {
        val a     = Input(UInt(w.W))
        val b     = Input(UInt(w.W))
        val load  = Input(Bool())
        val out   = Output(UInt(w.W))
        val valid = Output(Bool())
    })
    val x = Reg(UInt())
    val y = Reg(UInt())
    val state = RegInit(GCDState.empty)

    io.valid := state === GCDState.done
    when ((state === GCDState.empty) || (state === GCDState.done)) {
        when (io.load) {
            x := io.a
            y := io.b
            state := GCDState.running
        }
    } .elsewhen (state === GCDState.running) {
        when (x > y) {
            x := x - y
        } .elsewhen (x <= y) {
            y := y - x
        }
        when (y === 0.U) {
            state := GCDState.done
            io.valid := true.B
        }
    }
    printf(p"in: ${io.a} ${io.b} load: ${io.load}, reg: $x $y, state: $state\n")
    io.out := x
}

>You will verify the GCD module by adding assertions to the `GCDChecker` module (which inherits from GCD). We have added some assumptions to remove uninteresting corner cases. Add assertions to check the following:
>1. The final result is a factor of both inputs. In other words, the result when GCD terminates divides both inputs without producing a remainder.
>2. While the GCD module is executing (in the `running` state), at least one of the register values (`x` or `y`) decreases.

In [ ]:
class GCDChecker(w: Int) extends GCD(w) {
    // Assume inputs are non-zero and stable
    assume(io.a =/= 0.U)
    assume(io.b =/= 0.U)
    assume(io.a === past(io.a))
    assume(io.b === past(io.b))

    // #1 - Does the final result divide both inputs?
    // YOUR CODE HERE
    ???

    // #2 - Makes forward progress (x or y decreases every cycle when running)
    // YOUR CODE HERE
    ???
}

In [ ]:
verify(new GCDChecker(3), Seq(BoundedCheck(10)))